# 🔮 PRISM Engine - Quick Start

This notebook gets you running in 60 seconds.

**Steps:**
1. Mount Google Drive
2. Set up paths
3. Run the engine
4. See results

## Step 1: Mount Drive & Setup

In [ ]:
# === SETUP - WORKS IN COLAB OR LOCALLY ===
import sys
from pathlib import Path

def find_prism_root():
    """Find prism-engine root - works in Colab or locally."""
    try:
        from google.colab import drive
        IN_COLAB = True
    except ImportError:
        IN_COLAB = False

    if IN_COLAB:
        drive.mount('/content/drive')
        candidates = [
            Path('/content/drive/MyDrive/prism-engine/prism-engine'),
            Path('/content/drive/MyDrive/prism-engine'),
            Path('/content/prism-engine'),
        ]
    else:
        # Local: check current dir, parent (if in Start/), etc.
        candidates = [
            Path('.').resolve(),
            Path('..').resolve(),  # If running from Start/
        ]

    for path in candidates:
        if (path / '05_engine' / 'lenses').exists():
            return path
    return Path('.').resolve()

PRISM_ROOT = find_prism_root()
sys.path.insert(0, str(PRISM_ROOT))

print(f"✓ PRISM_ROOT = {PRISM_ROOT}")

In [ ]:
# Verify the structure exists
import os

expected_folders = ['01_fetch', '02_data_raw', '03_cleaning', '04_data_clean', 
                    '05_engine', '06_output', '07_interpretation', '08_visualization']

print("Checking folder structure...")
for folder in expected_folders:
    path = PRISM_ROOT / folder
    status = "✓" if path.exists() else "✗"
    print(f"  {status} {folder}")

# Check for data
data_path = PRISM_ROOT / 'data' / 'raw'
if data_path.exists():
    csvs = list(data_path.glob('*.csv'))
    print(f"\n✓ Found {len(csvs)} CSV files in data/raw/")
else:
    print("\n✗ No data/raw/ folder found")

## Step 2: Load Data

In [ ]:
import pandas as pd
import numpy as np

# Try to load master_panel.csv or build it
master_path = PRISM_ROOT / 'data' / 'raw' / 'master_panel.csv'

if master_path.exists():
    panel = pd.read_csv(master_path, index_col=0, parse_dates=True)
    print(f"✓ Loaded master_panel.csv")
else:
    # Build from individual CSVs
    print("Building panel from individual CSVs...")
    raw_dir = PRISM_ROOT / 'data' / 'raw'
    
    dfs = {}
    for f in raw_dir.glob('*.csv'):
        if f.name != 'master_panel.csv':
            try:
                df = pd.read_csv(f, index_col=0, parse_dates=True)
                name = f.stem.upper()
                if len(df.columns) == 1:
                    dfs[name] = df.iloc[:, 0]
                else:
                    # Use first numeric column
                    for col in df.columns:
                        if df[col].dtype in ['float64', 'int64']:
                            dfs[name] = df[col]
                            break
            except:
                pass
    
    panel = pd.DataFrame(dfs)
    print(f"✓ Built panel from {len(dfs)} files")

print(f"\nPanel shape: {panel.shape}")
print(f"Date range: {panel.index[0]} to {panel.index[-1]}")
print(f"Columns: {list(panel.columns)}")

In [ ]:
# Quick look at the data
panel.tail()

In [ ]:
# Check for NaNs
nan_pct = (panel.isna().sum() / len(panel) * 100).round(1)
print("NaN % by column:")
print(nan_pct[nan_pct > 0].sort_values(ascending=False) if nan_pct.any() else "No NaNs!")

## Step 3: Clean Data

In [ ]:
# Simple cleaning: forward fill, backward fill, drop remaining NaNs
panel_clean = panel.ffill().bfill().dropna()

print(f"Before cleaning: {panel.shape}")
print(f"After cleaning:  {panel_clean.shape}")
print(f"NaNs remaining:  {panel_clean.isna().sum().sum()}")

## Step 4: Run a Single Lens (Test)

In [ ]:
# Test with the simplest lens first - Magnitude
try:
    from engine.lenses.magnitude_lens import MagnitudeLens
    print("Import style 1 worked")
except:
    try:
        from prism_engine.engine.lenses.magnitude_lens import MagnitudeLens  
        print("Import style 2 worked")
    except:
        # Direct import
        exec(open(PRISM_ROOT / '05_engine' / 'lenses' / 'magnitude_lens.py').read())
        print("Direct exec worked")

In [ ]:
# If imports are tricky, here's a universal loader
import importlib.util

def load_lens(lens_name):
    """Load a lens by name, handling path issues."""
    lens_path = PRISM_ROOT / '05_engine' / 'lenses' / f'{lens_name}_lens.py'
    
    if not lens_path.exists():
        raise FileNotFoundError(f"Lens not found: {lens_path}")
    
    spec = importlib.util.spec_from_file_location(lens_name, lens_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)
    
    # Find the lens class (assumes it ends with 'Lens')
    for name in dir(module):
        if name.endswith('Lens') and name != 'BaseLens':
            return getattr(module, name)
    
    raise ValueError(f"No Lens class found in {lens_name}")

print("✓ Lens loader ready")

In [ ]:
# Load and run Magnitude Lens
MagnitudeLens = load_lens('magnitude')
mag_lens = MagnitudeLens()

print("Running MagnitudeLens...")
mag_result = mag_lens.analyze(panel_clean)

print("\nResult keys:", list(mag_result.keys()))

if 'importance' in mag_result:
    imp = mag_result['importance']
    if isinstance(imp, pd.Series):
        print("\nTop 10 by Magnitude:")
        print(imp.sort_values(ascending=False).head(10))

## Step 5: Run Multiple Lenses

In [ ]:
# Run all available lenses
lens_names = ['magnitude', 'pca', 'influence', 'clustering', 'decomposition']

results = {}
for name in lens_names:
    try:
        print(f"Running {name}...", end=" ")
        LensClass = load_lens(name)
        lens = LensClass()
        results[name] = lens.analyze(panel_clean)
        print("✓")
    except Exception as e:
        print(f"✗ ({e})")

print(f"\nSuccessfully ran {len(results)}/{len(lens_names)} lenses")

## Step 6: Compare Results (Consensus)

In [ ]:
# Build consensus from all lenses
rankings = {}

for lens_name, result in results.items():
    if 'importance' in result:
        imp = result['importance']
        if isinstance(imp, pd.Series):
            # Convert to ranks (1 = most important)
            ranks = imp.rank(ascending=False)
            rankings[lens_name] = ranks

if rankings:
    # Combine into DataFrame
    rank_df = pd.DataFrame(rankings)
    
    # Average rank across lenses
    rank_df['avg_rank'] = rank_df.mean(axis=1)
    rank_df['std_rank'] = rank_df.std(axis=1)
    
    # Sort by average rank
    consensus = rank_df.sort_values('avg_rank')
    
    print("CONSENSUS RANKINGS (lower = more important)")
    print("="*60)
    print(consensus.head(15))

In [ ]:
# Visualize agreement between lenses
import matplotlib.pyplot as plt

if len(rankings) > 1:
    # Correlation matrix of rankings
    rank_df_lenses = pd.DataFrame(rankings)
    corr = rank_df_lenses.corr(method='spearman')
    
    plt.figure(figsize=(8, 6))
    plt.imshow(corr, cmap='RdYlGn', vmin=-1, vmax=1)
    plt.colorbar(label='Spearman Correlation')
    plt.xticks(range(len(corr)), corr.columns, rotation=45, ha='right')
    plt.yticks(range(len(corr)), corr.columns)
    plt.title('Lens Agreement Matrix')
    
    # Add correlation values
    for i in range(len(corr)):
        for j in range(len(corr)):
            plt.text(j, i, f'{corr.iloc[i, j]:.2f}', ha='center', va='center')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nAverage lens agreement: {corr.values[np.triu_indices_from(corr.values, 1)].mean():.2f}")

## Step 7: Save Results

In [ ]:
# Save consensus to output folder
output_dir = PRISM_ROOT / '06_output' / 'latest'
output_dir.mkdir(parents=True, exist_ok=True)

if 'consensus' in dir():
    consensus.to_csv(output_dir / 'consensus_rankings.csv')
    print(f"✓ Saved consensus to {output_dir}/consensus_rankings.csv")

# Save run info
import json
from datetime import datetime

run_info = {
    'timestamp': datetime.now().isoformat(),
    'data_shape': list(panel_clean.shape),
    'lenses_run': list(results.keys()),
    'top_5_indicators': list(consensus.head(5).index) if 'consensus' in dir() else []
}

with open(output_dir / 'run_info.json', 'w') as f:
    json.dump(run_info, f, indent=2)

print(f"✓ Saved run info to {output_dir}/run_info.json")

---

## 🎉 You Did It!

**What just happened:**
1. Loaded your data
2. Cleaned NaNs
3. Ran multiple mathematical lenses
4. Computed consensus rankings
5. Visualized lens agreement
6. Saved results

**Next steps:**
- Try more lenses: `'wavelet'`, `'network'`, `'regime_switching'`, `'tda'`
- Run validation tests
- Explore disagreements between lenses